In [1]:
# Parameters to set to run this notebook

# Initialize helpers.
import numpy as np
import project_path
from model.data_accessor import get_data_file_abs_path
from model.neuron_metadata import *
import model.init_conds as init_conds
neuron_metadata_collection = NeuronMetadataCollection.load_from_chem_json(get_data_file_abs_path('chem.json'))
N = neuron_metadata_collection.get_size()

key_neuron = "AWAR"
key_neuron_id = neuron_metadata_collection.get_id_from_name(key_neuron)
current_nA = 6
step_duration_s = 10

# AWAR fixed points at current_nA = 3 {'9952.56': 0, '10435.27': 3, '10461.02': 5}
# At current_nA = 6 {'20893.82': 0, '19900.50': 1, '19819.50': 3}

In [2]:
from model.neural_model import NeuralModel
from util.plot_util import *
import os
import pickle

def gen_dyns(current_nA, step_duration_s, init_cond_seed):
  model = NeuralModel(neuron_metadata_collection)
  model.init_conds = init_conds.generate_init_cond_with_seed(
    model, key_neuron, current_nA, init_cond_seed)
  model.set_I_ext_constant_currents({key_neuron: current_nA})
  model.init_kunert_2017()

  simul_timesteps = int(step_duration_s / model.dt)
  (v_mat, s_mat, v_normalized_mat) = model.run(simul_timesteps)
  return v_mat

final_v_to_init_cond_i = {}
for init_cond_i in [0,1,3]: 
  print("-------------------")
  print("Iteration " + str(init_cond_i))
  v_mat = gen_dyns(current_nA, step_duration_s, init_cond_i)
  key_neuron_stable_v = v_mat[-1, key_neuron_id]

  normed_v = "%.2f" % key_neuron_stable_v
  if normed_v not in final_v_to_init_cond_i:
    print("New normed v found! %s in i = %d" % (normed_v, init_cond_i))
    final_v_to_init_cond_i[normed_v] = init_cond_i
  print("Final %s mV = %.2f" % (key_neuron, key_neuron_stable_v))
  print(final_v_to_init_cond_i)

-------------------
Iteration 0
Timestep 0 out of 2000
Timestep 100 out of 2000
Timestep 200 out of 2000
Timestep 300 out of 2000
Timestep 400 out of 2000
Timestep 500 out of 2000
Timestep 600 out of 2000
Timestep 700 out of 2000
Timestep 800 out of 2000
Timestep 900 out of 2000


KeyboardInterrupt: 

In [ ]:
model = NeuralModel(neuron_metadata_collection)
model.init_kunert_2017()
v_mat = gen_dyns(
  current_nA = 6,
  step_duration_s = 20,
  init_cond_seed = 0)
key_neuron_stable_v = v_mat[-1, key_neuron_id]
print("Final %s mV = %.2f" % (key_neuron, key_neuron_stable_v))
fig = plot_potentials(
  neuron_names_to_show = [key_neuron],
  dynamics = v_mat,
  is_normalized_v = False,
  dt = model.dt,
  neuron_metadata_collection = neuron_metadata_collection)
fig.show()